# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_with_intrinsic_reward_V0 import * #PPO_Agent_v2  PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-03-28 11:24:54.550300: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 47195


2024-03-28 11:24:55.942955: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:47195/ (Press CTRL+C to quit)


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\BipedalWalker\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/BipedalWalker/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_ppo/' # Linux

ENV = "BipedalWalker-v3"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<BipedalWalker<BipedalWalker-v3>>>>

In [4]:
s = env.observation_space.sample()
s

array([-1.4566026 ,  1.2366161 , -1.3154955 ,  1.3200742 ,  1.9649619 ,
       -1.1010382 ,  0.52088755,  0.15153418, -1.2581639 ,  0.44086   ,
        0.8954138 , -1.3308717 ,  0.0916764 ,  0.6512324 , -2.016662  ,
       -0.5096049 , -0.93953717, -0.05540914, -0.9809887 ,  0.69532627,
        1.6089568 ,  2.3321304 , -0.64416   , -1.2997762 ], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box(-inf, inf, (24,), float32), (24,))

In [6]:
s = env.reset()
s

array([ 2.7475364e-03, -3.9825372e-06,  3.0979741e-04, -1.5999973e-02,
        9.2095330e-02, -4.0882971e-04,  8.6019182e-01,  1.7053873e-03,
        1.0000000e+00,  3.2502260e-02, -4.0879875e-04,  8.5373855e-01,
        2.8912639e-04,  1.0000000e+00,  4.4081402e-01,  4.4582012e-01,
        4.6142277e-01,  4.8955020e-01,  5.3410280e-01,  6.0246104e-01,
        7.0914888e-01,  8.8593185e-01,  1.0000000e+00,  1.0000000e+00],
      dtype=float32)

In [7]:
s = env.reset()
env.action_space, env.action_space, env.step(s)

(Box(-1.0, 1.0, (4,), float32),
 Box(-1.0, 1.0, (4,), float32),
 (array([-5.3773387e-03, -2.3405088e-04, -4.6622381e-03, -1.8318227e-03,
          4.3742281e-01,  8.1091568e-02,  1.1204553e-01, -9.9342249e-02,
          1.0000000e+00,  3.4027529e-01,  1.1910359e-01,  1.1373204e-01,
         -1.5025806e-01,  1.0000000e+00,  4.4963509e-01,  4.5474136e-01,
          4.7065625e-01,  4.9934652e-01,  5.4479069e-01,  6.1451679e-01,
          7.2333956e-01,  9.0366006e-01,  1.0000000e+00,  1.0000000e+00],
        dtype=float32),
  -0.3732849320993683,
  False,
  {}))

### Training/Hyperparam run

In [8]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "logs_hyper/fit_PPO/"

In [9]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo"
    

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo/", evaluation_epoch = env._max_episode_steps, training_steps = 700000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.90, discount_max = 0.99, 
                #discount= 0.99,
                #gae_factor = 0.95, 
                gae_min = 0.85, gae_max = 0.96, 
                policy_clip =0.2,
                lr_actor_crit_min = 0.00001, lr_actor_crit_max = 0.0001,
                #entropy_factor = 0.05, 
                entropy_factor_min = 0.01, entropy_factor_max = 0.1,
                lr_model_max = None, kl_divergence_target = None,#0.1,#
                #dense_layers = [128,128],
                dense_min = 32, dense_max = 256,
                environment_name=ENV, max_num_layers_act = 2,
                num_layers_model = None, training_epoch = 50,
                memory_size = env._max_episode_steps, 
                normalize_reward=False, normalize_advantage= False,
                scaling_factor_reward = 0.1
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "logs_hyper/ppo/V0_"#Training/fit_ppo/"

        training_steps = 1000000
        entropy_factor = 0.05
        discount = 0.99
        dense_units_actor = [256,256]#64, 32]
        num_layers_actor = 2
        dense_units_critic = [256,256]#64,32]
        num_layers_crit =2
        num_layer_m = 1
        dense_units_model = [64]
        model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, dense_units_model,  num_layers_actor, num_layers_crit, num_layer_m, writer, 
                      environment_name = ENV,reward_scaler = 1, return_agent = True, lr_actor= 0.00001, lr_critic= 0.00001, 
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50, id = 1, scaling_factor_reward = 0.1)
 
 

Trial 2 Complete [00h 00m 00s]

Best total_train_reward So Far: None
Total elapsed time: 00h 00m 00s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.91              |0.96              |discount
0.85              |0.91              |gae_lambda
5.774e-05         |4.0193e-05        |lr_actor_critic
0.096956          |0.076689          |entropy_coeff
2                 |1                 |n_dense_layers_actor
200               |99                |dense_units_act_crit_0
0.49137           |0.14206           |kl_divergence_target
77                |None              |dense_units_act_crit_1



Traceback (most recent call last):
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/mnt/ECB0DA94B0DA649C/Artificial_Intelligence/Portfolio/Reinforcement Learning Projects/RL/BipedalWalker/../PPO_Agent_with_intrinsic_reward_V0.py", line 

Trial number :  3


RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/n/anaconda3/envs/RL_env/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/mnt/ECB0DA94B0DA649C/Artificial_Intelligence/Portfolio/Reinforcement Learning Proj

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=10):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"

for trials in tuner.oracle.get_best_trials(num_trials=50):
    if int(trials.trial_id) in [6,11,15,16,23,24,26]:
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
        training_steps = 2000000
        entropy_factor = trials.hyperparameters.values["entropy_coeff"]
        discount = trials.hyperparameters.values["discount"]
        n_dense_layers = trials.hyperparameters.values["n_dense_layers"]


        dense_units =  [trials.hyperparameters.values['dense_units_'+str(i)] for i in range(n_dense_layers)]
        gae = trials.hyperparameters.values["gae_lambda"]
        lr = trials.hyperparameters.values["learning_rate"]
        policy_clip = trials.hyperparameters.values["policy_clip"]

        model = run_training(training_steps,  discount = discount, dense_units = dense_units, num_layers = n_dense_layers, writer = writer, 
                        environment_name = ENV,reward_scaler = 1, return_agent = True, lr= lr, sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                        evaluation_epoch = env._max_episode_steps, gae_lambda = gae, entropy_coeff = entropy_factor, 
                        policy_clip = policy_clip, training_epoch = 50, memory_size= env._max_episode_steps, id = int(trials.trial_id))
        
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))


In [ ]:
final_rewards